# Практикческая работа №8
На тему: "Методы защиты от атак на модели ИИ"
<br>Выполнил студент группы ББМО-02-23
<br> Беляев Степан Константинович

## Цель задания:
Изучить методы защиты моделей ИИ от различных атак, включая методы защиты на уровне данных,
моделирования и обучения. Реализовать эти методы и проверить их эффективность против атак,
изученных ранее.


### Шаг 1. Защита с помощью Adversarial Training

Adversarial Training — это метод защиты, который заключается в том, чтобы обучать модель на
противоречивых примерах. Этот метод помогает модели научиться быть более устойчивой к атакам,
так как она сталкивается с противоречивыми примерами на этапе обучения

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets import mnist
# Загрузка данных MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
# Нормализация данных
train_images = train_images / 255.0
test_images = test_images / 255.0
# Преобразование меток в one-hot encoding
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)
# Функция FGSM атаки
def fgsm_attack(image, epsilon, gradient):
  perturbation = epsilon * np.sign(gradient)
  adversarial_image = image + perturbation
  adversarial_image = np.clip(adversarial_image, 0, 1)
  return adversarial_image
# Функция для генерации противоречивых примеров
def generate_adversarial_examples(model, images, labels, epsilon):
  adversarial_images = []
  for i in range(len(images)):
    image = tf.convert_to_tensor(images[i].reshape((1, 28, 28)))
    label = labels[i]
    if len(label.shape) > 1 and label.shape[1] > 1:
        label = np.argmax(label),
    label = tf.convert_to_tensor(label)
    with tf.GradientTape() as tape:
      tape.watch(image)
      prediction = model(image)
      loss = tf.keras.losses.categorical_crossentropy(label[None], prediction)
    gradient = tape.gradient(loss, image)
    adversarial_image = fgsm_attack(image.numpy(), epsilon, gradient.numpy())
    adversarial_images.append(adversarial_image.reshape(28, 28))
  return np.array(adversarial_images)
# Создание модели
def create_model():
  model = tf.keras.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')])
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  return model
# Обучение модели с противоречивыми примерами
def adversarial_training(model, train_images, train_labels, epsilon):
 for epoch in range(5): # Короткое обучение для демонстрации
  print(f'Epoch:{epoch}')
  for i in range(0, len(train_images), 64):
    batch_images = train_images[i:i+64]
    batch_labels = train_labels[i:i+64]
    # Генерация противоречивых примеров для текущей партии данных
    adversarial_images = generate_adversarial_examples(model, batch_images, batch_labels, epsilon)
    # Объединение оригинальных и противоречивых примеров
    combined_images = np.concatenate([batch_images, adversarial_images], axis=0)
    combined_labels = np.concatenate([batch_labels, batch_labels], axis=0)
    # Обучение на комбинированных данных
    model.train_on_batch(combined_images, combined_labels)
# Инициализация модели
model = create_model()
# Тренировка модели с защитой на противоречивых примерах
adversarial_training(model, train_images[:1500], train_labels[:1500], epsilon=0.1) # ограничил тренировочные данные для ускорения обучения
# Сохранение защищенной модели
model.save('adversarial_trained_model.h5')

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch:0


Epoch:1
Epoch:2
Epoch:3
Epoch:4


### Шаг 2: Градиентная маскировка (Gradient Masking)
Gradient Masking — это метод защиты, который затрудняет доступ к градиентам модели для атак. Он
используется для уменьшения информации, доступной для атакующих, и усложнения поиска
направленных изменений.


In [2]:
# Для демонстрации мы можем использовать специальные функции активации
from tensorflow.keras.layers import Activation
# Обновление модели для градиентной маскировки
def create_masked_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu'),
 tf.keras.layers.Dense(10), Activation('softplus') # Используем softplus вместо softmax для градиентной маскировки
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
 return model
# Обучение модели с градиентной маскировкой
masked_model = create_masked_model()
masked_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с градиентной маскировкой
masked_model.save('masked_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.8751 - loss: 0.4649
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9621 - loss: 0.1303
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9749 - loss: 0.0850
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9813 - loss: 0.0639
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9851 - loss: 0.0504


### Шаг 3: Регуляризация и нормализация для повышения устойчивости

In [3]:
# Модель с регуляризацией и нормализацией
def create_regularized_model():
 model = tf.keras.Sequential([
 tf.keras.layers.Flatten(input_shape=(28, 28)),
 tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)),
 tf.keras.layers.Dropout(0.5),
 tf.keras.layers.BatchNormalization(),
 tf.keras.layers.Dense(10, activation='softmax')
 ])
 model.compile(optimizer='adam', loss='categorical_crossentropy',
metrics=['accuracy'])
 return model
# Обучение модели с регуляризацией и нормализацией
regularized_model = create_regularized_model()
regularized_model.fit(train_images, train_labels, epochs=5)
# Сохранение модели с регуляризацией
regularized_model.save('regularized_model.h5')

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8028 - loss: 1.3087
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 5ms/step - accuracy: 0.8700 - loss: 0.6226
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - accuracy: 0.8659 - loss: 0.6140
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - accuracy: 0.8587 - loss: 0.6285
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8618 - loss: 0.6114


### Шаг 4: Оценка моделей на противоречивых примерах

In [4]:
# Загрузка атакованной модели
protected_model = tf.keras.models.load_model('adversarial_trained_model.h5')
# Генерация противоречивых примеров для тестовых данных
adversarial_test_images = generate_adversarial_examples(protected_model, test_images[1000:2000], test_labels[1000:2000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, test_labels[1000:2000])
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.4482 - loss: 1.5858
Accuracy of protected model on adversarial examples: 0.4560000002384186


Модель, обученная на противоречивых примерах, показала неудавлетворительный результат с точнсть в 44%.

Проверим также её точность на атакованных данных, используемых при обучении

In [5]:
adversarial_test_images = generate_adversarial_examples(protected_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = protected_model.evaluate(adversarial_test_images, train_labels[:1000])
print(f'Accuracy of protected model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6799 - loss: 1.0502
Accuracy of protected model on adversarial examples: 0.6710000038146973


Точность составила 65%. Это гораздо лучше, однако следует проверить, возможно ли можно её повысить, увеличив время обучения и выборку.

In [6]:
base_model = create_model()
base_model.fit(train_images, train_labels, epochs=5)
# --- Оценка моделей на противоречивых примерах ---
test_adversarial_images = generate_adversarial_examples(base_model, test_images[:3000], test_labels[:3000], epsilon=0.1)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.8776 - loss: 0.4308
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9639 - loss: 0.1190
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - accuracy: 0.9767 - loss: 0.0797
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9831 - loss: 0.0573
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 8s 4ms/step - accuracy: 0.9873 - loss: 0.0425


In [7]:
# Оценка защищенных моделей
print("Evaluation of models on adversarial examples:")
print("Base Model Accuracy on Adversarial Examples:")
base_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Adversarially Trained Model Accuracy on Adversarial Examples:")
protected_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Masked Model Accuracy on Adversarial Examples:")
masked_model.evaluate(test_adversarial_images, test_labels[:3000])
print("Regularized Model Accuracy on Adversarial Examples:")
regularized_model.evaluate(test_adversarial_images, test_labels[:3000])

Evaluation of models on adversarial examples:
Base Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0775 - loss: 6.7621
Adversarially Trained Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7060 - loss: 0.9092
Masked Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2115 - loss: 4.5774
Regularized Model Accuracy on Adversarial Examples:
94/94 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6736 - loss: 1.1073


[1.1522610187530518, 0.6636666655540466]

Как видно, модель, обученная на противоречивых примерах, и модель с регуляризацией и нормализацией показали неплохой результат. Лучший принадлежит модели на противорчеивых данных. Около 70%. А модель с маскировкой спраивлась гораздо хуже. Лишь 20%.

Противоречивые примеры были сгенерированы на базовой модели. Посмотрим, как поведут себя модели, если противоречивые примеры будут сгенерированы конкретно для них


In [8]:
adversarial_test_images = generate_adversarial_examples(masked_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = masked_model.evaluate(adversarial_test_images, train_labels[:1000])
print(f'Accuracy of masked model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1323 - loss: 5.8703 
Accuracy of masked model on adversarial examples: 0.13600000739097595


In [9]:
adversarial_test_images = generate_adversarial_examples(regularized_model, train_images[:1000], train_labels[:1000], epsilon=0.1)
# Оценка защищенной модели на противоречивых примерах
test_loss, test_acc = regularized_model.evaluate(adversarial_test_images, train_labels[:1000])

print(f'Accuracy of regularized model on adversarial examples: {test_acc}')

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.4818 - loss: 1.6310
Accuracy of regularized model on adversarial examples: 0.47099998593330383


Исходя из тестов, наиболее успешными себя показали модель, обученная на противоречивых примерах, и модель с регуляризацией и нормализацией, хотя полученное значение точности все равно довольно низкое